## Centrality

Degree of centrality:
https://neo4j.com/blog/graph-algorithms-neo4j-degree-centrality/

In [ ]:
def print_degree_centrality_given_year(tx,year):
    for record in tx.run("MATCH (a:CountryYear) "
                         "RETURN a.country as name, a.year as year, size((a)-[:RESIDE_IN]->()) AS originates_from, size((a)<-[:RESIDE_IN]-()) AS resides_in ".format(year)): 
        
        print(", ".join([record["name"],str(record["year"]),str(record["originates_from"]),str(record["resides_in"])]))

In [ ]:
year = 2017

with driver.session() as session:
    session.read_transaction(print_degree_centrality_given_year,year)

In [ ]:
MATCH (u:User)
RETURN u.id AS name,
size((u)-[:FOLLOWS]->()) AS follows,
size((u)<-[:FOLLOWS]-()) AS followers

plot histogram of node degree

In [ ]:
CALL algo.betweenness.stream(
'MATCH (n:CountryYear) WHERE n.year=2009 RETURN id(n) AS id',
'MATCH (n)-[r:RESIDE_IN]->(m) RETURN id(n) AS source, id(m) AS target',
{graph: "cypher"})

YIELD nodeId, centrality

MATCH (CountryYear) WHERE id(CountryYear) = nodeId
RETURN CountryYear.countryearId AS countryyear,centrality
ORDER BY centrality DESC;

In [ ]:
CALL algo.betweenness.stream(
'MATCH (n:CountryYear) WHERE n.year=2009 RETURN id(n) AS id',
'MATCH (n)-[r:RESIDE_IN]->(m) WHERE NOT EXISTS(r.affected_idps) RETURN id(n) AS source, id(m) AS target',
{graph: "cypher"})

YIELD nodeId, centrality

MATCH (n:CountryYear) WHERE id(n) = nodeId
RETURN n.countryearId AS countryyear, centrality
ORDER BY centrality DESC

## Comunity Detection